# Checking for GPU Support
Getting GPU support for TensorFlow installed on your Windows system can be difficult. Articles such as *TensorFlow + Jupyter Notebook + Nvidia DIY Setup* (https://towardsdatascience.com/tensorflow-jupyter-notebook-nvidia-diy-setup-473acfe5b0e) make life easier for Linux developers. With this in mind, this section talks about the process that appears to work best for getting GPU support installed on Windows (**all the code works equally well on Windows and Linux systems**):

1. Install Visual Studio 2019 Community Edition.
2. Install CUDA Toolkit 11.3.
3. Install cuDNN 11.3.
4. Set the required paths, even if you perform a default installation, because otherwise no one can find the CUDA Toolkit.
5. Check for pip and install if necessary.
6. Install Anaconda 2020.07.
7. Create a conda environment. Specifically install Python 3.8.3 (Python 3.9.x doesn't work).
8. Activate the new environment.
9. Use pip install --upgrade tensorflow-gpu to install TensorFlow. Accept whatever version of TensorFlow pip selects. Don't use conda to install TensorFlow, it installs a version that uses Intel's oneAPI version, which doesn't use the local GPU.
10. Test the installation.  Yay!  It works!

Here are some notes on this steps that will prove helpful:

- It's helpful to know whether you have GPU support for creating models. Otherwise, it could take a very long time to complete the model. Trying to configure such a setup can prove time consuming and error prone. You need to know which versions of the CUDA Toolkit and cuDNN to install for your particular GPU. This process starts by determining the compute compatibility level of your GPU using the resource at https://developer.nvidia.com/cuda-gpus. For example, an NVidia GeForce GTX 760 (chosen for a specific reason in this example) has a compute capacity of 3.0.

- Once you determine the compute capacity, you must find a CUDA Toolkit version that will actually work, which means cross-referencing one to the other. Unfortunately, finding such information on the NVidia site is nearly impossible, so use the Wikipedia article at https://en.wikipedia.org/wiki/CUDA instead. Even though the article shows that a GPU with a compute capacity of 3.0 should work find with the 10.2 CUDA Toolkit version, in practice, it's actually better to use the 9.2 version, which you can download at https://developer.nvidia.com/cuda-toolkit-archive.

- Now you have a CUDA Toolkit in hand, so it's time to look for a compatible cuDNN version. You can find it at https://developer.nvidia.com/rdp/cudnn-archive.

- In addition, even though few sources tell you to do so, you must include the locations of all of these elements as part of your path statement, such as for a Windows system:

`
SET PATH=C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.0\bin;%PATH%
SET PATH=C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.0\extras\CUPTI\lib64;%PATH%
SET PATH=C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v11.0\include;%PATH%
SET PATH=C:\tools\cuda\bin;%PATH%`

- Creating a Anaconda Channel (environment) comes next. Before you assume that you can use the latest version of TensorFlow, which requires a compute compatibilty of 3.5, you might want to check around for articles like the one at https://medium.com/@chrzkan/install-tensorflow-gpu-with-cuda-9-2-driver-410-x-on-ubuntu-16-04-c4a5a09e0ace. In this case, you find out that you must use TensorFlow version 1.12 instead to support your older GPU. So, to create your environment in this case, you type **`conda create -n TF-GPU python=3.6 anaconda=2020.07 tensorflow-gpu=1.12`** at the Anaconda Prompt and press Enter. Of course, if you're not using this specific version of the CUDA Toolkit, then you use the versions needed for your GPU. Of note in this case is that TensorFlow 1.12 won't run on Python 3.8, so you must use version 3.6 instead. Once the installation is done, you can activate your new environment by typing **`conda activate TF-GPU`** at the Anaconda Prompt and press Enter.

- Finally, at this point, you can type **`python`** at the Anaconda Prompt and press Enter to test some code. The following check determines if TensorFlow can actually see your GPU. It has to be able to see the GPU before it can use it. This particular check works for both TensorFlow 1.0 and 2.0. **This check works equally well on Linux and Windows systems.**

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from tensorflow.python.client import device_lib 
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 1494365489555324758,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 4181262336
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10884665388966003667
 physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1660 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5"]

## TensorFlow 1.0
The following code is designed to work with TensorFlow 1.0.

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
tf.test.is_gpu_available()

In [ ]:
with tf.device('/GPU:0'):
    a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
    c = tf.matmul(a, b)

# Show how the calculation is performed.
print(c)

# Show the calculation result.
with tf.Session() as sess:
    print(c.eval())

## TensorFlow 2.0
The following code is designed to work with TensorFlow 2.0. It won't work with TensorFlow 1.0.

In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)
